# Import ELK Snapshots onto a Fabric slice without the Measurement Framework

### Prerequisite
- A running fabric slice (For example: [Create single node backup Slice](./create_single_node_backup.ipynb))
- An ELK snapshot tar file to import

#### General Imports

In [ ]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

#### Get your slice information
*Set slice_name and node_name below before running*

In [ ]:
slice_name='backup-testing'
node_name='data-backup'


slice = fablib.get_slice(name=slice_name)
slice.show();
node = slice.get_node(name=node_name)
node.show();

# Instructions to start ELK and import your snapshot data

#### Ensure Docker and Docker-Compose are installed on the node

In [ ]:
commands = [
    'sudo apt-get update',
    'sudo apt-get install docker -y',
    'sudo apt-get install docker-compose -y',
]
try:
    for command in commands:
        stdout, stderr = node.execute(command)
except Exception as e:
    print(f"Fail: {e}")

#### Print Docker/Docker-Compose versions

In [ ]:
stdout, stderr = node.execute('sudo docker -v')
stdout, stderr = node.execute('sudo docker-compose -v') 

#### Set up NAT64 Server
We need to do this step so that we can clone the Github repo with our docker app.

In [ ]:
commands = [
    # TODO: This line shouldn't need to be done.. temporary fix?
    f"sudo sed -i '1s/^/0.0.0.0 {node_name}\\n/' /etc/hosts",
    "sudo sed -i '/nameserver/d' /etc/resolv.conf",
    "sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf'",
    "sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf'",
    "sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'"
]
try:
    for command in commands:
        stdout, stderr = node.execute(command)
except Exception as e:
    print(f"Fail: {e}")

#### Create docker container and import your data

In [ ]:
commands = [
    'sudo git clone https://github.com/jackhancockuky/elk-docker-container.git'
]
try:
    for command in commands:
        stdout, stderr = node.execute(command)
except Exception as e:
    print(f"Fail: {e}")

#### Upload a snapshot file into ELK

In [ ]:
# Enter your elk snapshot tar file name (must be in data_transfer_service/snapshots directory)
snapshot_tar_name = "elk_backup_2.tar"


node.upload_file(f"./snapshots/{snapshot_tar_name}", f'/tmp/{snapshot_tar_name}')
stdout, stderr = node.execute(f'sudo mv /tmp/{snapshot_tar_name} /home/ubuntu/elk-docker-container/snapshots/{snapshot_tar_name}') 

#### View snapshots ready to be imported into ELK

In [ ]:
stdout, stderr = node.execute('ls elk-docker-container/snapshots') 

#### Import a snapshot

In [ ]:
# Uncomment below and change name to a different file from above if you'd like to
# snapshot_tar_name = "elk_backup_2.tar"

In [ ]:
stdout, stderr = node.execute('sudo tar -xvf elk-docker-container/snapshots/elk_backup_2.tar -C elk-docker-container/imported_data') 

#### Start the docker compose app

In [ ]:
stdout, stderr = node.execute('sudo docker-compose -f elk-docker-container/docker-compose.yml up -d') 

#### Register the repository

In [ ]:
stdout, stderr = node.execute('curl -X PUT -H "Content-Type: application/json" -d \'{ "type": "fs", "settings": { "location": "/usr/share/elasticsearch/imported_data/_data", "compress": true } }\' http://localhost:9200/_snapshot/imported_repository')

#### View repository after registration

In [ ]:
stdout, stderr = node.execute('curl -X GET "http://localhost:9200/_cat/snapshots/imported_repository?v"')

#### Delete all indices and restore snapshot (replace snapshot name and change index list if you want to customize)

In [ ]:
stdout, stderr = node.execute('curl -X DELETE "http://localhost:9200/_all"')
stdout, stderr = node.execute('curl -X POST "http://localhost:9200/_snapshot/imported_repository/snapshot_test/_restore?pretty" -H "Content-Type: application/json" -d \'{ "indices": "*", "index_settings": { "index.number_of_replicas": 1 } }\'')

### Your data is now imported on the node.

----
### View the index list

In [ ]:
stdout, stderr = node.execute('curl -X GET "http://localhost:9200/_cat/indices?pretty"')

----
### View data from Kibana

#### On your local terminal create a tunnel with:

'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}'

#### Then go to http://localhost:10030
----

### To reuse slice with new data (if you are done then just delete slice)
#### Take down and reset container to original state:

In [ ]:
#stdout, stderr = node.execute('docker-compose down')
#stdout, stderr = node.execute('docker volume rm elk-docker-container_es-data')

#### Remove backup data

In [ ]:
#stdout, stderr = node.execute('rm -rf elk-docker-container/imported_data/*')

#### Start with steps again